In [9]:
# %% Importing libraries
import pymongo
import pandas as pd

from utils.db_utils import MongoUtils

mongo_config = MongoUtils()
myclient = mongo_config.get_mongo_conn()

MONGO_DB = mongo_config.mongo_db
MONGO_COLLECTION = mongo_config.mongo_collection

mydb = myclient[MONGO_DB]
col_all = mydb[MONGO_COLLECTION]
# %%
myquery = {"root.property.overview.state": "NY"}

docs = col_all.find(myquery)

import pprint

sample_doc = col_all.find_one()
pprint.pprint(sample_doc)

# save sample doc to json
from json import loads, dumps, dump
json_doc = loads(dumps(list(sample_doc)))

import json

# Assuming you have already created the json_doc as shown in your code

# Open a file for writing
with open('sample_doc.json', 'w', encoding='utf-8') as json_file:
    # Write the json_doc to the file
    json.dump(json_doc, json_file, ensure_ascii=False, indent=4)



{'_id': ObjectId('6408d30441c2b087bf0c789c'),
 'root': {'extractTimestamp': '08/03/2023 13:21:30',
          'hash_md5': '467e0efd1db0590898c29407',
          'property': {'compNearbyHomes': {'comps': [{'bathrooms': 6,
                                                      'bedrooms': 5,
                                                      'city': 'Malibu',
                                                      'coverPhoto': 'https://photos.zillowstatic.com/fp/9a818e6500802e9c29f5973885c6892b-p_c.jpg',
                                                      'currency': 'USD',
                                                      'hdpUrl': '/homedetails/26731-Via-Linda-St-Malibu-CA-90265/20554747_zpid/',
                                                      'homeStatus': 'FOR_SALE',
                                                      'homeType': 'SINGLE_FAMILY',
                                                      'isPremierBuilder': False,
                                              

In [26]:
sample_doc
# json_doc = loads(dumps(list(sample_doc)))
json.dump(sample_doc['root'], open('sample_doc.json', 'w'), indent=4)

In [4]:
docs[0]

{'_id': ObjectId('6408c57c41c2b087bf0c7897'),
 'root': {'hash_md5': '2ea58e49ffa0c476ce143b36',
  'extractTimestamp': '07/03/2023 21:29:45',
  'zpid': 2094308893,
  'property': {'overview': {'utcScrapeTime': '2023-03-08 01:40:56.525147',
    'zpid': 2094308893,
    'streetAddress': '5 Beekman St #47B',
    'city': 'New York',
    'state': 'NY',
    'price': 4000000,
    'homeStatus': 'FOR_SALE',
    'yearBuilt': 2015,
    'isRentalListingOffMarket': False,
    'contingentListingType': None,
    'timeOnZillow': '118 days',
    'pageViewCount': 203,
    'favoriteCount': 16,
    'daysOnZillow': 118,
    'hdpUrl': 'zillow.com//homedetails/5-Beekman-St-47B-New-York-NY-10038/2094308893_zpid/',
    'desktopWebHdpImageLink': 'https://photos.zillowstatic.com/fp/e44b323c35a85b9f6f47e5be6033eead-p_h.jpg',
    'propertyTaxRate': 0.88,
    'lotSize': None},
   'location': {'streetAddress': '5 Beekman St #47B',
    'city': 'New York',
    'state': 'NY',
    'stateId': 43,
    'zipcode': '10038',
   

In [10]:
def unnest_dicts(dicts_list):
    def unnest_dict(d, prefix=''):
        unnested = {}
        for k, v in d.items():
            new_key = f'{prefix}.{k}' if prefix else k
            new_key
            if isinstance(v, dict):
                unnested.update(unnest_dict(v, new_key))
            else:
                unnested[new_key] = v
        return unnested

    unnested_dicts = [unnest_dict(d) for d in dicts_list]
    return unnested_dicts


def clean_json(string_list):
    container = []
    for string in string_list:
        string = string.replace("'", '"')
        string = string.replace("None","null")
        
        container.append(string)
    return container

doc_list = unnest_dicts(docs)
len(doc_list)
#%%
df = pd.DataFrame(doc_list)
df = df.loc[:, ~df.columns.str.contains('coreLogic')]
# sort by root.extractTimestamp
df = df.sort_values(by=['root.extractTimestamp'], ascending=False)
df = df.drop_duplicates(subset=['root.zpid'])
# drop all columns that include 'coreLogic' 


ss = df['root.property.pricing.sellingSoon'].to_list()
print(ss)
percentiles = []
for i in ss:
    try:
        percentile = i[0]['percentile']
        percentiles.append(percentile)
       
    except Exception as e:
        percentile = None
        percentiles.append(percentile)
df['sellingSoon'] = percentiles
        

[[{'treatmentId': 'model_0', 'percentile': 0.32}], [{'treatmentId': 'model_0', 'percentile': 0.12}], [{'treatmentId': 'model_0', 'percentile': 0.62}], [{'treatmentId': 'model_0', 'percentile': 0.65}], [{'treatmentId': 'model_0', 'percentile': 0.93}], [{'treatmentId': 'model_0', 'percentile': 0.28}], [{'treatmentId': 'model_0', 'percentile': 0.44}], [{'treatmentId': 'model_0', 'percentile': 0.51}], [{'treatmentId': 'model_0', 'percentile': 0.4}], [{'treatmentId': 'model_0', 'percentile': 0.69}], [{'treatmentId': 'model_0', 'percentile': 0.75}], [{'treatmentId': 'model_0', 'percentile': 0.27}], [{'treatmentId': 'model_0', 'percentile': 0.47}], [{'treatmentId': 'model_0', 'percentile': 0.9}], [{'treatmentId': 'model_0', 'percentile': 0.36}], [{'treatmentId': 'model_0', 'percentile': 0.77}], [{'treatmentId': 'model_0', 'percentile': 0.64}], [{'treatmentId': 'model_0', 'percentile': 0.27}], [{'treatmentId': 'model_0', 'percentile': 0.13}], [{'treatmentId': 'model_0', 'percentile': 0.63}], [

### Lets goooo

found zillows model output

In [11]:
#df_in = pd.read_csv('/Users/darien/Library/Mobile Documents/com~apple~CloudDocs/PropertizeAI Research/EDA/Initial-EDA-MongoDB/data/nyc_unique_properties_unnested2.csv')
import json

schools = df['root.property.schools.schools'].to_list()
df.head(10)

,_id,root.hash_md5,root.extractTimestamp,root.zpid,root.property.overview.utcScrapeTime,root.property.overview.zpid,root.property.overview.streetAddress,root.property.overview.city,root.property.overview.state,root.property.overview.price,...,root.property.propertyFeatures.zoning,root.property.propertyFeatures.zoningDescription,root.property.propertyFeatures.hasRentControl,root.property.propertyFeatures.hasHomeWarranty,root.property.propertyFeatures.isNewConstruction,root.property.propertyFeatures.structureType,root.scrapeDate,root.scrapeTime,root.property.overview.zipcode,sellingSoon
42581,6420e8caf8624e45c6c43bb0,ded5798761495d13b461db24,27/03/2023 00:52:26,122274768,2023-03-26 18:26:00.171945,122274768,157 W 57th St PENTHOUSE 88,New York,NY,39000000,...,None,None,NaN,False,False,None,27/03/2023,00:52:26,NaN,0.32
42580,6420e8b2f8624e45c6c43baf,5b72134271681b116d2cc8ee,27/03/2023 00:52:02,2073108029,2023-03-26 18:26:00.171945,2073108029,14 E 69th St,New York,NY,26500000,...,None,None,NaN,False,False,None,27/03/2023,00:52:02,NaN,0.12
42579,6420e8b1f8624e45c6c43bae,ab55429b82de7f91f75d47f7,27/03/2023 00:52:01,244731102,2023-03-26 18:26:00.171945,244731102,102 Bradhurst Ave APT 1107,New York,NY,599999,...,None,None,NaN,False,False,None,27/03/2023,00:52:01,NaN,0.62
42578,6420e8b1f8624e45c6c43bad,435756f6908dfefdd94a8826,27/03/2023 00:52:01,31518482,2023-03-26 18:26:00.171945,31518482,50 Central Park W,New York,NY,17990000,...,None,None,NaN,False,False,None,27/03/2023,00:52:01,NaN,0.65
42577,6420e8aff8624e45c6c43bac,d93ef60b21f0daad9792e518,27/03/2023 00:51:59,31500522,2023-03-26 18:26:00.171945,31500522,22 Bank St,New York,NY,26000000,...,None,None,NaN,False,False,None,27/03/2023,00:51:59,NaN,0.93
42576,6420e8aff8624e45c6c43bab,a0f89cb447f677cb5ea2364e,27/03/2023 00:51:58,31533511,2023-03-26 18:26:00.171945,31533511,610 Park Ave APT 16E,New York,NY,24850000,...,None,None,NaN,False,False,None,27/03/2023,00:51:58,NaN,0.28
42575,6420e8adf8624e45c6c43baa,ebfb8a68910fd09959456fb7,27/03/2023 00:51:57,219674337,2023-03-26 18:26:00.171945,219674337,432 Park Ave APT 66A,New York,NY,24800000,...,None,None,NaN,False,False,None,27/03/2023,00:51:57,NaN,0.44
42574,6420e8acf8624e45c6c43ba9,1e084900e7aa434b5cf1f80f,27/03/2023 00:51:56,2068566679,2023-03-26 18:26:00.171945,2068566679,50 United Nations Plz #DPH42,New York,NY,39950000,...,None,None,NaN,False,False,None,27/03/2023,00:51:56,NaN,0.51
42573,6420e8a8f8624e45c6c43ba8,2139d09f8bbfe6b6f9bc8c04,27/03/2023 00:51:52,72538873,2023-03-26 18:26:00.171945,72538873,115 Central Park W APT 19EF,New York,NY,24500000,...,None,None,NaN,False,False,None,27/03/2023,00:51:52,NaN,0.40
42572,6420e8a7f8624e45c6c43ba7,61c7dfb64cda9b1fb0e66adf,27/03/2023 00:51:51,32331409,2023-03-26 18:26:00.171945,32331409,278 Hull Ave,Staten Island,NY,630000,...,R3-1,None,NaN,False,False,None,27/03/2023,00:51:51,NaN,0.69


In [ ]:
results_container = []
avg_school_rating = []
for listing in schools:
    if listing is None:
        results_container.append([None, None, None])
        
        continue
    total = 0
    total_count = 0
    distance = 0
    distance_count = 0
    private_count = 0
    
    for school in listing:
        if school.get('rating') is not None:
            total += school.get('rating')
            total_count += 1
        if school.get('distance') is not None:
            distance += school.get('distance')
            distance_count += 1
        if school.get('type') == 'Private':
            private_count += 1
    if total_count == 0:
        t = None
    else: t = round(total/total_count,4)
    if distance_count == 0:
        d = None
    else: d = round(distance/distance_count,4)
    avg_school_rating.append(t)
    results_container.append([t, d , total_count, private_count])
#%%
results_container

In [16]:
schoolDf = pd.DataFrame(
    results_container,
    columns=["avgRating", "avgDistance", "schoolCount", "privateSchoolCount"],
)
df_schools_add = df.join(schoolDf)
df = df.drop(["avgRating", "avgDistance", "schoolCount", "privateSchoolCount"], axis=1)
df["avgSchoolRating"] = avg_school_rating

df.to_csv('NYC_4_schools.csv')

In [38]:
df1 = pd.read_csv("NYC_4_schools.csv")
edit = df1["root.property.propertyFeatures.hoaFee"].to_list()
edit_container = []

for x in edit:
    if x is None:
        edit_container.append(None)
        continue

    if x == "nan":
        edit_container.append(None)
        continue
    x = f"{x}"

    x = x.replace("$", "")
    x = x.replace(",", "")
    x = x.replace("HOA Fee: ", "")
    x = x.replace("monthly", "")
    x = x.replace(" ", "")

    edit_container.append(x)
edit_container


df1["root.property.propertyFeatures.hoaFee"] = edit_container
df1["root.property.propertyFeatures.hoaFee"] = pd.to_numeric(
    df1["root.property.propertyFeatures.hoaFee"], errors="coerce"
)
df1.to_csv("NYC_5_schools.csv")

/var/folders/m0/l8z7qyb90rn8h8_3vx43nsl00000gn/T/ipykernel_95526/557877964.py:1: DtypeWarning: Columns (12,50,51,52,53,54,55,72,76,77,79,80,82,84,93,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('NYC_4_schools.csv')


In [ ]:
avgPriceChange = []
for i in df["root.property.pricing.priceHistory"].to_list():
    if i is None:
        avgPriceChange.append(None)
        continue
    price_change = 0
    count = 0
    for j in i:
        if j.get("priceChangeRate") is not None:
            price_change += j.get("priceChangeRate")
            count += 1
    if count == 0:
        avgPriceChange.append(None)
    else:
        avgPriceChange.append(round(price_change / count, 4))

avgPriceChange

In [19]:
df['avgPriceChange'] = avgPriceChange
df.head()

,_id,root.hash_md5,root.extractTimestamp,root.zpid,root.property.overview.utcScrapeTime,root.property.overview.zpid,root.property.overview.streetAddress,root.property.overview.city,root.property.overview.state,root.property.overview.price,...,root.property.propertyFeatures.zoningDescription,root.property.propertyFeatures.hasRentControl,root.property.propertyFeatures.hasHomeWarranty,root.property.propertyFeatures.isNewConstruction,root.property.propertyFeatures.structureType,root.scrapeDate,root.scrapeTime,root.property.overview.zipcode,sellingSoon,avgPriceChange
42581,6420e8caf8624e45c6c43bb0,ded5798761495d13b461db24,27/03/2023 00:52:26,122274768,2023-03-26 18:26:00.171945,122274768,157 W 57th St PENTHOUSE 88,New York,NY,39000000,...,None,NaN,False,False,None,27/03/2023,00:52:26,NaN,0.32,0.0162
42580,6420e8b2f8624e45c6c43baf,5b72134271681b116d2cc8ee,27/03/2023 00:52:02,2073108029,2023-03-26 18:26:00.171945,2073108029,14 E 69th St,New York,NY,26500000,...,None,NaN,False,False,None,27/03/2023,00:52:02,NaN,0.12,-0.0116
42579,6420e8b1f8624e45c6c43bae,ab55429b82de7f91f75d47f7,27/03/2023 00:52:01,244731102,2023-03-26 18:26:00.171945,244731102,102 Bradhurst Ave APT 1107,New York,NY,599999,...,None,NaN,False,False,None,27/03/2023,00:52:01,NaN,0.62,0.0000
42578,6420e8b1f8624e45c6c43bad,435756f6908dfefdd94a8826,27/03/2023 00:52:01,31518482,2023-03-26 18:26:00.171945,31518482,50 Central Park W,New York,NY,17990000,...,None,NaN,False,False,None,27/03/2023,00:52:01,NaN,0.65,88464.9898
42577,6420e8aff8624e45c6c43bac,d93ef60b21f0daad9792e518,27/03/2023 00:51:59,31500522,2023-03-26 18:26:00.171945,31500522,22 Bank St,New York,NY,26000000,...,None,NaN,False,False,None,27/03/2023,00:51:59,NaN,0.93,0.6655


In [107]:
schools = [s.replace("'", '"') for s in schools]
s_list = []
for listing in schools:
    try:

        input_string = listing.replace("None", "null")
        l = json.loads(input_string)
        s_list.append(l)
    except:
        s_list.append(None)

In [108]:
s_list[0][0]
results_container = []
for listing in s_list:
    if listing is None:
        results_container.append([None, None, None])
        continue
    total = 0
    total_count = 0
    distance = 0
    distance_count = 0
    private_count = 0

    for school in listing:
        if school.get("rating") is not None:
            total += school.get("rating")
            total_count += 1
        if school.get("distance") is not None:
            distance += school.get("distance")
            distance_count += 1
        if school.get("type") == "Private":
            private_count += 1
    if total_count == 0:
        t = None
    else:
        t = total / total_count
    if distance_count == 0:
        d = None
    else:
        d = distance / distance_count

    results_container.append([t, d, total_count, private_count])

results_container[0]
schoolDf = pd.DataFrame(
    results_container,
    columns=["avgRating", "avgDistance", "schoolCount", "privateSchoolCount"],
)
#
df_in = df_in.join(schoolDf)

df_in.head()

,_id,root.hash_md5,root.extractTimestamp,root.zpid,root.property.overview.utcScrapeTime,root.property.overview.zpid,root.property.overview.streetAddress,root.property.overview.city,root.property.overview.state,root.property.overview.price,...,root.property.pictures.staticMap,root.property.pictures.streetViewMetadataUrlMediaWallLatLong,root.property.pictures.streetViewMetadataUrlMediaWallAddress,root.property.pictures.streetViewTileImageUrlMediumLatLong,root.property.pictures.streetViewTileImageUrlMediumAddress,root.property.pictures.streetViewServiceUrl,avgRating,avgDistance,schoolCount,privateSchoolCount
0,642086086dcfe806ff9b7d7f,6ea459b0b170a32594b03a91,26/03/2023 17:51:04,2073149629,07:49.9,2073149629,107-16 37th Avenue UNIT 3D,Corona,NY,555000,...,['https://maps.googleapis.com/maps/api/staticm...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://proxy.zillowapi.com/street-view-url?zp...,4.000000,0.766667,3.0,0.0
1,642086086dcfe806ff9b7d7e,52d4f53fa20b0b96b35c2fc7,26/03/2023 17:51:04,72541532,07:49.9,72541532,348 W 56th St APT 4A,New York,NY,549000,...,['https://maps.googleapis.com/maps/api/staticm...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://proxy.zillowapi.com/street-view-url?zp...,5.000000,1.866667,3.0,0.0
2,642086086dcfe806ff9b7d7d,a41bfdfd56fdda7f5506f289,26/03/2023 17:51:04,2059276930,07:49.9,2059276930,91-20 139th Street,Jamaica,NY,549999,...,['https://maps.googleapis.com/maps/api/staticm...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://proxy.zillowapi.com/street-view-url?zp...,4.666667,0.600000,3.0,0.0
3,642086086dcfe806ff9b7d7c,c4d878c81c21ac82f44f7428,26/03/2023 17:51:04,29837565,07:49.9,29837565,4181 De Reimer Ave,Bronx,NY,555000,...,['https://maps.googleapis.com/maps/api/staticm...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://proxy.zillowapi.com/street-view-url?zp...,5.666667,0.600000,3.0,0.0
4,642086086dcfe806ff9b7d7b,8e1dfb20f75d160fa8afc7df,26/03/2023 17:51:03,2060407922,07:49.9,2060407922,35-20 Leverich St #703,Jackson Heights,NY,555000,...,['https://maps.googleapis.com/maps/api/staticm...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://maps.googleapis.com/maps/api/streetvie...,https://proxy.zillowapi.com/street-view-url?zp...,7.666667,0.566667,3.0,0.0


In [113]:
# create a column that is the difference between 'root.property.pricing.zestimate' and 'root.property.pricing.price' in df_in dataframe
df_in['selling_soon'] = df_in['root.property.pricing.sellingSoon']
df_in['zestimateDiff'] = df_in['root.property.pricing.zestimate'] - df_in['root.property.pricing.price']

tax_history = df_in['root.property.pricing.taxHistory']
tax_history = clean_json(tax_history)
tax_history = [json.loads(t) for t in tax_history]

avgTaxIncrease = []
for i in tax_history:
    print()
    valueIncreaseRate = 0
    valueIncreaseCount = 0
    for j in i:
        try:
            if j.get('valueIncreaseRate') is not None:
                valueIncreaseRate += j.get('valueIncreaseRate')
                valueIncreaseCount += 1
        except:
            pass
    if valueIncreaseCount == 0:
        valueIncreaseRate = None
    else:
        valueIncreaseRate = valueIncreaseRate/valueIncreaseCount
    avgTaxIncrease.append(valueIncreaseRate)

df_in['avgTaxIncrease'] = avgTaxIncrease


In [116]:
df_in.to_csv("NYC_unique_3_school_tax_add.csv", index=False)

In [85]:
price_history = df_in['root.property.pricing.priceHistory']

price_history = [price_hist.replace("None", "null") for price_hist in price_history]
price_history = [price_hist.replace("'", '"') for price_hist in price_history]

In [99]:
price_history = [hist.replace("'", '"') for hist in price_history]
price_history = [r'{}'.format(h) for h in price_history]
price_history = [price_hist.replace("None", "null") for price_hist in price_history]
price_hist = []
for i in price_history:
    try:
        load = json.loads(i)
        price_hist.append(load)
    except:
        price_hist.append(None)